In [39]:
# AI 서비스 실행절차(필요함수 선언)

# 사진파일 리스트 읽어오기
def select_file(path_dir):
    import os
    file_dir = os.listdir(path_dir)
    print('해당되는 파일을 골라주세요.')
    for i in range(len(file_dir)):
        print(f'{i}번 : {file_dir[i]}')
    file_num = int(input())
    return file_dir[file_num]

# # 해당되는 템플릿 선택하기
# def select_template(template_path):s
#     import pandas as pd
#     import numpy as np
#     template = pd.read_csv(template_path, sep = ' ')
#     print('해당되는 템플릿을 골라주세요.')
#     for i in range(len(template['name'])):
#         print(f'{i}번 : {template["name"][i]}')
#     template_num = int(input())
#     return template['id'][template_num]

# NaverAPI서버에 요청
def request_NaverOCR(key_path, file_name):
    import requests
    import uuid
    import time
    import json

    api_url = key_path[key_path['title'] == 'InvokeUrl']['content'][0]
    secret_key = key_path[key_path['title'] == 'SecretKey']['content'][1]

    image_file = f'C:\\Workspace\\python\\빅데이터 지능형서비스 개발 팀프로젝트\\PJTJ4U\\조정범\\Data\\input\\{file_name}'
    output_file = f'C:\\Workspace\\python\\빅데이터 지능형서비스 개발 팀프로젝트\\PJTJ4U\\조정범\\Data\\output\\{file_name.split(".")[0]+".json"}'

    request_json = {
        'images': [
            {
                'format': 'jpg',
                'name': 'demo',
                # 'templateIds': [int(template_id)]
            }
        ],
        'requestId': str(uuid.uuid4()),
        'version': 'V2',
        'timestamp': int(round(time.time() * 1000))
    }

    payload = {'message': json.dumps(request_json).encode('UTF-8')}
    files = [
    ('file', open(image_file,'rb'))
    ]
    headers = {
    'X-OCR-SECRET': secret_key
    }

    response = requests.request("POST", api_url, headers=headers, data = payload, files = files)

    res = json.loads(response.text.encode('utf8'))
    # print(res)

    with open(output_file, 'w', encoding='utf-8') as outfile:
        json.dump(res, outfile, indent=4, ensure_ascii=False)
    return output_file

def extract_info(output_file):
    import json
    import pandas as pd
    import numpy as np
    with open(output_file, 'r', encoding = 'utf-8') as f:
        json_data = json.load(f)

    ctg_list = []
    contents_list = []

    for i in range(len(json_data['images'][0]['fields'])):
        ctg_list.append(json_data['images'][0]['fields'][i]['name'])
        contents_list.append(json_data['images'][0]['fields'][i]['inferText'])
    df_info = pd.DataFrame([contents_list], columns = ctg_list)
    df_info['보내는분 주소'] = df_info['보내는분 주소'].apply(lambda x: x.split('\n')[0])
    df_info['받는분 주소'] = df_info['받는분 주소'].apply(lambda x: x.split('\n')[0])
    return pd.DataFrame(df_info)

# 도로명 주소 받아오기
def juso_addr(df_info, juso_api_key):
    import pandas as pd
    import numpy as np
    import requests
    from urllib.parse import quote
    road_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do'
    addr_list = []
    with open(juso_api_key) as f:
        api_key = f.read()
    df_info['도로명주소(보내는분)'] = [0 for _ in range(len(df_info['보내는분 주소']))]
    df_info['도로명주소(받는분)'] = [0 for _ in range(len(df_info['받는분 주소']))]
    for i in range(len(df_info['보내는분 주소'])):
        try:
            option = f'confmKey={api_key}&currentPage=1&countPerPage=10&keyword={quote(df_info["보내는분 주소"][i])}' # f스트링에서 &을 통해 연결
            url = f'{road_url}?{option}&resultType=json'
            result = requests.get(url).json()
            addr = result['results']['juso'][0]['roadAddr']    
            df_info['도로명주소(보내는분)'][i] = addr
        except:
            print(i)
    for i in range(len(df_info['받는분 주소'])):
        try:
            option = f'confmKey={api_key}&currentPage=1&countPerPage=10&keyword={quote(df_info["받는분 주소"][i])}' # f스트링에서 &을 통해 연결
            url = f'{road_url}?{option}&resultType=json'
            result = requests.get(url).json()
            addr = result['results']['juso'][0]['roadAddr']    
            df_info['도로명주소(받는분)'][i] = addr
        except:
            print(i)
    df_info1 = df_info.copy()
    return df_info1

# 카카오 API활용 위도,경도 받아오기
def location_point(df_info1, kakao_api_key):
    import pandas as pd
    import numpy as np
    import requests
    from urllib.parse import quote
    search_url = 'https://dapi.kakao.com/v2/local/search/address.json'
    df_info = df_info1.copy()
    with open(kakao_api_key) as f:
        api_key2 = f.read()
    lat_list, lng_list = [], []
    for i in range(len(df_info['도로명주소(보내는분)'])):
        try:
            url = f'{search_url}?query={quote(df_info["도로명주소(보내는분)"][i])}'
            result = requests.get(url,
                            headers = {'Authorization': f'KakaoAK {api_key2}'}).json()
            lng = float(result['documents'][0]['x']) # 경도
            lng_list.append(lng)
            lat = float(result['documents'][0]['y']) # 위도
            lat_list.append(lat)
        except:
            lng, lat = 0, 0
            lng_list.append(lng)
            lat_list.append(lat)
    df_info['위도(보내는분)'] = lat_list
    df_info['경도(보내는분)'] = lng_list

    lat_list, lng_list = [], []
    for i in range(len(df_info['도로명주소(받는분)'])):
        try:
            url = f'{search_url}?query={quote(df_info["도로명주소(받는분)"][i])}'
            result = requests.get(url,
                            headers = {'Authorization': f'KakaoAK {api_key2}'}).json()
            lng = float(result['documents'][0]['x']) # 경도
            lng_list.append(lng)
            lat = float(result['documents'][0]['y']) # 위도
            lat_list.append(lat)
        except:
            lng, lat = 0, 0
            lng_list.append(lng)
            lat_list.append(lat)
    df_info['위도(받는분)'] = lat_list
    df_info['경도(받는분)'] = lng_list
    df_info2 = df_info.copy()
    return df_info2

# 지도에 위치표시(보내는분, 받는분)
def point_map(df_info2, zoom_size):
    import folium
    df_info = df_info2.copy()
    location = [(df_info['위도(보내는분)'].mean() + df_info['위도(받는분)'].mean()) / 2, (df_info['경도(보내는분)'].mean() + df_info['경도(받는분)'].mean()) / 2]
    map = folium.Map(location = location, zoom_start = zoom_size)
    map
    for i in range(len(df_info['도로명주소(보내는분)'])):
        folium.Marker(
            location = [df_info['위도(보내는분)'][i], df_info['경도(보내는분)'][i]], # 위도, 경도
            popup =  folium.Popup(f'보내는사람 : {df_info["보내는분 성명"][i]}', max_width = 200),
            tooltip = f'주소 : {df_info["도로명주소(보내는분)"][i]}',
            icon = folium.Icon(color = 'blue', info = 'info-sign'),
        ).add_to(map)
    for i in range(len(df_info['도로명주소(받는분)'])):
        folium.Marker(
            location = [df_info['위도(받는분)'][i], df_info['경도(받는분)'][i]], # 위도, 경도
            popup =  folium.Popup(f'보내는사람 : {df_info["받는분 성명"][i]}', max_width = 200),
            tooltip = f'주소 : {df_info["도로명주소(받는분)"][i]}',
            icon = folium.Icon(color = 'red', info = 'info-sign'),
        ).add_to(map)
    return map

In [40]:
# 필요패키지 로드
import pandas as pd
import numpy as np

# 파일선택 후 API를 요청하여 추출정보 받아오기
path_dir = 'C:\\Workspace\\python\\빅데이터 지능형서비스 개발 팀프로젝트\\PJTJ4U\\조정범\\Data\\input'
template_path = 'C:\\Workspace\\python\\빅데이터 지능형서비스 개발 팀프로젝트\\PJTJ4U\\조정범\\Data\\path\\Template(Naver_OCR).txt'
key_path = pd.read_csv('C:\\Workspace\\python\\빅데이터 지능형서비스 개발 팀프로젝트\\PJTJ4U\\조정범\\Data\\path\\Key_URL(Naver_OCR).txt', sep = '_')
file_name = select_file(path_dir)
juso_api_key = 'C:\\Workspace\\python\\Data_Science\\04.공공데이터활용\\roadapikey.txt'
kakao_api_key = 'C:\\Workspace\\python\\Data_Science\\04.공공데이터활용\\kakaoapikey.txt'

# template_id = select_template(template_path)
output_file = request_NaverOCR(key_path, file_name)
df_info = extract_info(output_file)
df_info1 = juso_addr(df_info, juso_api_key)
df_info2 = location_point(df_info1, kakao_api_key)
point_map(df_info2, 11)

해당되는 파일을 골라주세요.
0번 : 경동택배_샘플.jpg
1번 : 우체국(제너럴테스트).jpg
2번 : 우체국(테스트용).jpg
3번 : 우체국1.jpg
4번 : 우체국_샘플.jpg
5번 : 우체국_샘플2.jpg
6번 : 천일택배_샘플.jpg
7번 : 한진택배_샘플.jpg


<ipython-input-39-4cde077672e6>:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_info['도로명주소(보내는분)'][i] = addr
<ipython-input-39-4cde077672e6>:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_info['도로명주소(받는분)'][i] = addr
